In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [13]:
# Memastikan penggunaan GPU jika tersedia
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Device Name:", torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

CUDA Available: True
GPU Device Name: NVIDIA GeForce RTX 2070 SUPER
Using device: cuda


In [14]:
# 1. Load Dataset
# Penjelasan: Membaca dataset dari file CSV.
data = pd.read_csv('E:\Bot\MLD\heart.csv')

<>:3: SyntaxWarning: invalid escape sequence '\B'
<>:3: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Mizan\AppData\Local\Temp\ipykernel_12316\241991062.py:3: SyntaxWarning: invalid escape sequence '\B'
  data = pd.read_csv('E:\Bot\MLD\heart.csv')


In [15]:
# 2. Preprocessing
# Penjelasan: Memisahkan fitur (X) dan target (y), serta melakukan normalisasi dataset.
X = data.drop(columns=['target']).values
y = data['target'].values

# Normalisasi fitur menggunakan StandardScaler.
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Membagi data menjadi training dan testing set (80% train, 20% test).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mengubah data ke dalam bentuk Tensor.
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

# Membuat DataLoader untuk pelatihan dan pengujian.
batch_sizes = [16, 32, 64, 128, 256, 512]
train_datasets = [DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True) for batch_size in batch_sizes]
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

In [16]:
# 3. Definisi Model
# Penjelasan: Membuat arsitektur MLP dengan jumlah layer dan neuron yang dapat dikonfigurasi.
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, neurons_per_layer, activation_fn):
        super(MLP, self).__init__()
        layers = []

        # Input Layer
        layers.append(nn.Linear(input_size, neurons_per_layer))
        layers.append(activation_fn)

        # Hidden Layers
        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(activation_fn)

        # Output Layer (2 kelas: 0 dan 1)
        layers.append(nn.Linear(neurons_per_layer, 2))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [17]:
# 4. Pengaturan Eksperimen
hidden_layers_options = [1, 2, 3]
neurons_options = [4, 8, 16, 32, 64]
activation_functions = {
    'linear': nn.Identity(),
    'sigmoid': nn.Sigmoid(),
    'relu': nn.ReLU(),
    'softmax': nn.Softmax(dim=1),
    'tanh': nn.Tanh()
}
epochs_options = [1, 10, 25, 50, 100, 250]
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]

# Fungsi pelatihan model

def train_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    train_loss_history = []
    test_loss_history = []
    accuracy_history = []

    model = model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        # Loop setiap batch pada data training
        for batch in train_loader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass dan optimasi
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Evaluasi pada data testing
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        train_loss_history.append(train_loss / len(train_loader))
        test_loss_history.append(test_loss / len(test_loader))
        accuracy_history.append(correct / total)

    return train_loss_history, test_loss_history, accuracy_history

# Loop Eksperimen
results = []

for hidden_layers in hidden_layers_options:
    for neurons in neurons_options:
        for activation_name, activation_fn in activation_functions.items():
            for epochs in epochs_options:
                for lr in learning_rates:
                    for train_loader in train_datasets:
                        model = MLP(X_train.shape[1], hidden_layers, neurons, activation_fn)
                        criterion = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=lr)

                        # Latih model
                        train_loss, test_loss, accuracy = train_model(model, train_loader, test_loader, criterion, optimizer, epochs)

                        # Simpan hasil
                        results.append({
                            'hidden_layers': hidden_layers,
                            'neurons': neurons,
                            'activation': activation_name,
                            'epochs': epochs,
                            'learning_rate': lr,
                            'batch_size': train_loader.batch_size,
                            'accuracy': accuracy[-1]
                        })

# Konversi hasil eksperimen ke DataFrame
results_df = pd.DataFrame(results)
results_df.to_csv('experiment_results.csv', index=False)

print("Eksperimen selesai. Hasil disimpan di 'experiment_results.csv'.")

Eksperimen selesai. Hasil disimpan di 'experiment_results.csv'.
